In [13]:
import pandas as pd

# Load the data
df = pd.read_excel('../data/legislatives-2022-1er-tour.xlsx')

In [14]:
print(df.keys())

Index(['Code du département', 'Libellé du département',
       'Code de la circonscription', 'Libellé de la circonscription',
       'Code de la commune', 'Libellé de la commune', 'Code du b.vote',
       'Inscrits', 'Abstentions', '% Abs/Ins',
       ...
       'Unnamed: 187', 'Unnamed: 188', 'Unnamed: 189', 'Unnamed: 190',
       'Unnamed: 191', 'Unnamed: 192', 'Unnamed: 193', 'Unnamed: 194',
       'Unnamed: 195', 'Unnamed: 196'],
      dtype='object', length=197)


In [73]:
import pandas as pd

# Define the base column names
base_columns = ['Code du département', 'Libellé du département', 'Code de la circonscription', 'Libellé de la circonscription', 'Code de la commune', 'Libellé de la commune', 'Code du b.vote', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot']
candidate_columns = ['N�Panneau', 'Sexe', 'Nom', 'Prénom', 'Nuance', 'Voix', '% Voix/Ins', '% Voix/Exp']

# Initialize an empty list to store the rows
data = []

# Initialize a variable to store the maximum number of candidates
max_candidates = 0

# Open the file and read it line by line
with open('../data/leg_2022-1er-tour.txt', 'r', encoding='latin1') as f:
    for line in f:
        # Split the line into elements
        elements = line.strip().split(';')
        
        # The first 22 elements are the voting location information
        location_info = elements[:22]
        
        # The remaining elements are candidate information
        candidate_info = elements[22:]
        
        # Update max_candidates if this line has more candidates
        max_candidates = max(max_candidates, len(candidate_info)//8)
        
        # Initialize an empty list to store the candidate information
        candidate_data = []
        
        # Loop through the candidate information, 7 elements at a time
        for i in range(0, len(candidate_info), 8):
            # Append the candidate information to the candidate_data list
            candidate_data.extend(candidate_info[i:i+8])
        
        # Combine the location information with the candidate information and append it to the data list
        data.append(location_info + candidate_data)
print(max_candidates)
# Create the column names
columns = base_columns + [f'{col}_{i}' for i in range(1, max_candidates + 2) for col in candidate_columns]

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=columns)

21


In [74]:
print(df.keys())

Index(['Code du département', 'Libellé du département',
       'Code de la circonscription', 'Libellé de la circonscription',
       'Code de la commune', 'Libellé de la commune', 'Code du b.vote',
       'Inscrits', 'Abstentions', '% Abs/Ins',
       ...
       '% Voix/Ins_21', '% Voix/Exp_21', 'N�Panneau_22', 'Sexe_22', 'Nom_22',
       'Prénom_22', 'Nuance_22', 'Voix_22', '% Voix/Ins_22', '% Voix/Exp_22'],
      dtype='object', length=197)


In [96]:
# Initialise an dataframe with each circonscription, département and the number of votes for each candidate
results = pd.DataFrame(columns=['Code de la circonscription', 'Code du département'] + [f'Voix_{i}' for i in range(1, max_candidates + 2)] + [f'Parti_{i}' for i in range(1, max_candidates + 2)])

for index, row in df.iloc[1:].iterrows():
    # Get the circonscription and département information
    circonscription = row['Code de la circonscription']
    departement = row['Code du département']
    
    # Get the number of votes for each candidate
    votes = [int(row[f'Voix_{i}']) if row[f'Voix_{i}'] is not None else 0 for i in range(1, max_candidates + 2)]
    parties = [row[f'Nuance_{i}'] for i in range(1, max_candidates + 2)]

    # Find the row in the results DataFrame where 'Code de la circonscription' and 'Code du département' are equal
    result_index = results[(results['Code de la circonscription'] == circonscription) & (results['Code du département'] == departement)].index
    
    if len(result_index) > 0:
        # If the row exists, update it
        results.loc[result_index, [f'Voix_{i}' for i in range(1, max_candidates + 2)] + [f'Parti_{i}' for i in range(1, max_candidates + 2)]] = votes + parties
    else:
        # If the row doesn't exist, add a new row to the DataFrame
        new_row = [circonscription, departement] + votes + parties
        results.loc[len(results)] = new_row


KeyboardInterrupt: 

In [98]:
# Define a function to apply to each group
def process_group(group):
    # Get the number of votes for each candidate
    votes = [group[f'Voix_{i}'].apply(pd.to_numeric, errors='coerce').sum() if f'Voix_{i}' in group.columns else 0 for i in range(1, max_candidates + 2)]
    parties = [group[f'Nuance_{i}'].mode()[0] if f'Nuance_{i}' in group.columns and group[f'Nuance_{i}'].notna().any() else None for i in range(1, max_candidates + 2)]
    return pd.Series([group['Code de la circonscription'].iloc[0], group['Code du département'].iloc[0]] + votes + parties)

# Group the DataFrame and apply the function to each group
results = df.groupby(['Code de la circonscription', 'Code du département']).apply(process_group)

# Reset the index and set the column names
results.reset_index(drop=True, inplace=True)
results.columns = ['Code de la circonscription', 'Code du département'] + [f'Voix_{i}' for i in range(1, max_candidates + 2)] + [f'Parti_{i}' for i in range(1, max_candidates + 2)]



In [101]:
import numpy as np

# Define a function to find the winner for each group
def find_winner(row):
    # Get the votes and parties
    votes = row[[f'Voix_{i}' for i in range(1, max_candidates + 2)]].values
    parties = row[[f'Parti_{i}' for i in range(1, max_candidates + 2)]].values

    # Find the index of the maximum vote
    max_vote_index = np.argmax(votes)

    # Return the party with the maximum vote
    return parties[max_vote_index]

# Apply the function to each row of the DataFrame
results['Winner'] = results.apply(find_winner, axis=1)

# Count the number of groups won by each party
party_counts = results['Winner'].value_counts()

In [102]:
print(party_counts)

Winner
ENS       201
NUP       187
RN        110
LR         41
DVG        14
REG         9
DVD         9
DVC         3
DIV         1
DSV         1
UDI         1
Nuance      1
Name: count, dtype: int64
